In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install pytorch-lightning

     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 329 kB 60.9 MB/s 
     |████████████████████████████████| 132 kB 62.7 MB/s 
     |████████████████████████████████| 596 kB 34.9 MB/s 
     |████████████████████████████████| 829 kB 33.8 MB/s 
     |████████████████████████████████| 1.1 MB 33.6 MB/s 
     |████████████████████████████████| 271 kB 43.2 MB/s 
     |████████████████████████████████| 160 kB 54.0 MB/s 
     |████████████████████████████████| 192 kB 45.9 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=412e0ba1129cadcce23ad1bc438fa3d178a2fd40cdb1441632fd65a62c0c6c18
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found exis

In [ ]:
# tokenize and pad every document to make them of the same size
# this makes vectors from the documents
import os
import pandas as pd # importing libraries
import numpy as np
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.utils import shuffle


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from imblearn.over_sampling import RandomOverSampler
import torch
import torchmetrics
from torch.utils.data import DataLoader

import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping



seed_everything(42, workers=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Global seed set to 42


42

In [ ]:
def get_msr_data(paraphrase_data_path, data_part):
  # This function reads the MRPC dataset
  with open(os.path.join(paraphrase_data_path, data_part), 'r') as f:
    data = f.read()

  sentence_1_data = [el.split('\t')[3:][0] for el in data.split('\n')[1:-1]]
  sentence_2_data = [el.split('\t')[3:][1] for el in data.split('\n')[1:-1]]
  targets = [int(el.split('\t')[0][0]) for el in data.split('\n')[1:-1]]
  return sentence_1_data, sentence_2_data, targets

In [ ]:
def get_processed_df(dset, stop_words_l):
  documents_df = pd.DataFrame(dset, columns=['documents'])
  # removing special characters and stop words from the text
  documents_df['documents']=documents_df.documents.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )
  return documents_df

In [ ]:
def get_doc_embeddings(tokenized_paded_documents):
  # creating document-word embeddings
  document_word_embeddings=embedding_matrix[tokenized_paded_documents]
  # calculating average of word vectors of documents
  document_embeddings = document_word_embeddings.mean(1)
  return document_embeddings

In [ ]:
class EmbeddingData:
  
  def __init__(self, embs1, embs2, targets):
    self.embs1 = embs1
    self.embs2 = embs2
    self.targets = targets

  def __len__(self):
    return len(self.embs1)

  def __getitem__(self, idx):
    return {
        'emb1': self.embs1[idx],
        'emb2': self.embs2[idx],
        'target': self.targets[idx]
    }

class SiameseClassifier(pl.LightningModule):

    def __init__(self, learning_rate=1e-3):
        super().__init__()

        self.fc1 = torch.nn.Linear(300, 128)
        self.fc2 = torch.nn.Linear(128, 32)
        self.drop = torch.nn.Dropout(0.1)
        self.fc3 = torch.nn.Linear(64, 1)
        self.relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
        self.loss = torch.nn.BCELoss()
        self.accuracy_t = torchmetrics.Accuracy()
        self.accuracy_v = torchmetrics.Accuracy()
        self.learning_rate = learning_rate

    def forward(self, emb1, emb2):

        x1 = self.fc1(emb1)
        x2 = self.fc1(emb2)

        x1 = self.relu(x1)
        x2 = self.relu(x2)

        #x1 = self.drop(x1)
        #x2 = self.drop(x2)

        x1 = self.fc2(x1)
        x2 = self.fc2(x2)

        x1 = self.relu(x1)
        x2 = self.relu(x2)

        x = torch.cat([x1, x2], 1)
        
        x = self.fc3(x)
        x = self.sigmoid(x)

        return x

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20, eta_min=1e-5)
        return [optimizer], [lr_scheduler]

    def training_step(self, train_batch, batch_idx):
        x1, x2, y = train_batch.values()
        y = y.to(torch.float)
        y_hat = self.forward(x1, x2).squeeze(1)
        loss = self.loss(y_hat, y)
        preds = (y_hat > 0.5).to(torch.int)
        self.log('train_loss', loss)
        self.log('train_acc_step', self.accuracy_t(preds, y.to(torch.int)))
        return loss

    def validation_step(self, val_batch, batch_idx):
        x1, x2, y = val_batch.values()
        y = y.to(torch.float)
        y_hat = self.forward(x1, x2).squeeze(1)
        loss = self.loss(y_hat, y)
        preds = (y_hat > 0.5).to(torch.int)
        self.log('val_loss', loss)
        self.log('valid_acc_step', self.accuracy_v(preds, y.to(torch.int)))
        return loss
    
    def validation_epoch_end(self, validation_step_outputs):
        self.log('valid_acc_epoch', self.accuracy_v.compute())
        print('valid_acc_epoch', self.accuracy_v.compute())

    def training_epoch_end(self, outs):
        self.log('train_acc_epoch', self.accuracy_t.compute())
        print('train_acc_epoch', self.accuracy_t.compute())

In [ ]:
MAX_LEN = 128
EPOCHS = 100
batch_size = 64
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# loading pre-trained embeddings, each word is represented as a 300 dimensional vector
W2V_PATH= "/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz"
model_w2v= gensim.models.KeyedVectors.load_word2vec_format(W2V_PATH,binary=True)

In [ ]:
paraphrase_data_path = '/content/drive/MyDrive/msr_paraphrase'
train_1, train_2, train_targets = get_msr_data(paraphrase_data_path, 'msr_paraphrase_train.txt')
test_1, test_2, test_targets = get_msr_data(paraphrase_data_path, 'msr_paraphrase_test.txt')
train_1_2 = np.array([[el1, el2] for el1, el2 in zip(train_1, train_2)])
test_1_2 = np.array([[el1, el2] for el1, el2 in zip(test_1, test_2)])

ros = RandomOverSampler(random_state=0, sampling_strategy='minority')
train_1_2, train_targets = ros.fit_resample(train_1_2, train_targets)
ros = RandomOverSampler(random_state=0, sampling_strategy='minority')
test_1_2, test_targets = ros.fit_resample(test_1_2, test_targets)

test_1, test_2 = list(test_1_2[:,0]), list(test_1_2[:,1])
train_1, train_2 = list(train_1_2[:,0]), list(train_1_2[:,1])

In [ ]:
# Sample corpus
# The text will be enable the testing of this algorithm
stop_words_l = stopwords.words('english')
documents_df_train_1, documents_df_train_2, documents_df_test_1, documents_df_test_2 = (get_processed_df(dset, stop_words_l) for dset in (train_1, train_2, test_1, test_2))

In [ ]:
# tokenize and pad every document to make them of the same size
tokenizer = Tokenizer()
tokenizer.fit_on_texts(documents_df_train_1.documents + documents_df_train_2.documents)
tokenized_documents_train_1, tokenized_documents_train_2, tokenized_documents_test_1, tokenized_documents_test_2 = (tokenizer.texts_to_sequences(doc.documents) for doc in (documents_df_train_1, documents_df_train_2, documents_df_test_1, documents_df_test_2))
tokenized_doc_pad_train_1, tokenized_doc_pad_train_2, tokenized_doc_pad_test_1, tokenized_doc_pad_test_2 = (pad_sequences(doc,maxlen=MAX_LEN,padding='post') for doc in (tokenized_documents_train_1, tokenized_documents_train_2, tokenized_documents_test_1, tokenized_documents_test_2))
vocab_size=len(tokenizer.word_index)+1

In [ ]:
# creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 
embedding_matrix=np.zeros((vocab_size,300))
for word,i in tokenizer.word_index.items():
    if word in model_w2v:
        embedding_matrix[i]=model_w2v[word]

In [ ]:
'''import pickle
with open('/content/drive/MyDrive/w2v_model/tok_emb.pickle', 'wb') as f:
  pickle.dump((tokenizer, embedding_matrix), f)'''

In [ ]:
doc_emb_train_1, doc_emb_train_2, doc_emb_test_1, doc_emb_test_2 = (get_doc_embeddings(docs).astype(np.float32) for docs in (tokenized_doc_pad_train_1, tokenized_doc_pad_train_2, tokenized_doc_pad_test_1, tokenized_doc_pad_test_2))

In [ ]:
ds_train = EmbeddingData(doc_emb_train_1, doc_emb_train_2, train_targets)
ds_test = EmbeddingData(doc_emb_test_1, doc_emb_test_2, test_targets)

In [ ]:
train_loader = DataLoader(ds_train, batch_size=batch_size)
val_loader = DataLoader(ds_test, batch_size=batch_size)

# model
model = SiameseClassifier()

In [ ]:
early_stop_callback = EarlyStopping(monitor="valid_acc_epoch", min_delta=0.00, patience=15, verbose=False, mode="max")
checkpoint_callback = ModelCheckpoint(
    monitor="valid_acc_epoch",
    dirpath="/content/drive/MyDrive/w2v_model/",
    filename="emorec-{epoch:02d}-{valid_acc_epoch:.2f}",
    save_top_k=1,
    mode="max",
)

# training
trainer = pl.Trainer(accelerator=device, callbacks=[early_stop_callback, checkpoint_callback], max_epochs = EPOCHS, check_val_every_n_epoch=1)
trainer.fit(model, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name       | Type     | Params
----------------------------------------
0 | fc1        | Linear   | 38.5 K
1 | fc2        | Linear   | 4.1 K 
2 | drop       | Dropout  | 0     
3 | fc3        | Linear   | 65    
4 | relu       | ReLU     | 0     
5 | sigmoid    | Sigmoid  | 0     
6 | loss       | BCELoss  | 0     
7 | accuracy_t | Accuracy | 0     
8 | accuracy_v | Accuracy | 0     
----------------------------------------
42.7 K    Trainable params
0         Non-trainable params
42.7 K    Total params
0.171     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


valid_acc_epoch tensor(0.6875)


Training: 0it [00:00, ?it/s]

...

valid_acc_epoch tensor(0.5963)
train_acc_epoch tensor(0.5958)


Validating: 0it [00:00, ?it/s]

valid_acc_epoch tensor(0.5965)
train_acc_epoch tensor(0.5965)


Validating: 0it [00:00, ?it/s]

valid_acc_epoch tensor(0.5967)
train_acc_epoch tensor(0.5972)


In [ ]:
def predict_similarity(sent1, sent2):
  sent1, sent2 = re.sub(r'[^a-z ]', '', sent1.lower()), re.sub(r'[^a-z ]', '', sent2.lower())
  docs = tokenizer.texts_to_sequences([sent1, sent2])
  docs = pad_sequences(docs,maxlen=MAX_LEN,padding='post')
  embeddings = torch.Tensor(get_doc_embeddings(docs))
  pred = model(embeddings[0].unsqueeze(0), embeddings[1].unsqueeze(0)).detach().cpu().numpy()[0]
  result = 'Similar' if pred > 0.5 else 'Not Similar'
  return pred, result

In [ ]:
i = 76
print(test_1[i], '\n', test_2[i], '\n', test_targets[i])
sent1 = test_1[i]
sent2 = test_2[i]
predict_similarity(sent1, sent2)

From the start, however, the United States' declared goal was not just to topple Saddam but to stabilize Iraq and install a friendly government. 
 But the United States' ultimate goal was not just to topple Mr. Hussein but to stabilize the country and install a friendly government. 
 1


(array([0.7000551], dtype=float32), 'Similar')